In [1]:
!pip install transformers
!pip install googletrans==4.0.0-rc1
!pip install emoji==0.6.0
!pip install tweepy==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from googletrans import Translator
import torch
import tweepy
import time
import json
import numpy as np
import string 
import re 

punct = set(string.punctuation) 

In [3]:
def clear_text(text):
  try:
    text = re.sub("\$", "", text)
    text = re.sub("@\S+", "", text)
    text = re.sub("https?:\/\/.*[\r\n]*", "", text)
    text = "".join([ch for ch in text if ch not in punct])
    return text
  except:
    return text

In [4]:
def sentiment(sentences):
    translator = Translator()
    model_name = "finiteautomata/bertweet-base-sentiment-analysis"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    sentences_eng = translator.translate(sentences).text

    inputs = tokenizer(sentences_eng, padding=True, truncation=True, return_tensors = "pt")

    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    labels_predictions = torch.argmax(predictions, dim=1)
    labels = model.config.id2label
    return [{f"{labels[i.item()]}":p[i.item()].item()} for i,p in zip(labels_predictions,predictions)]


In [5]:
token = json.load(open('/content/drive/MyDrive/twitter_api.json'))

In [6]:

class Tweet(tweepy.StreamingClient):
    def __init__(self, bearer_token,n_tweets):
      super().__init__(bearer_token)
      self.limit = n_tweets
      self.count = 0
      pass

    def on_tweet(self,tweet):   
        # tweet_clear = clear_text(tweet.text)
        # print(f"{tweet_clear}")
        sentimentos = sentiment(tweet.text)   
        print(f"{tweet.text} - {sentimentos}")
        if self.count == self.limit:
          self.disconnect()
        else:
          self.count += 1
        
  

keyword  = 'bitcoin'
count = 0
printer = Tweet(token['bearer'],n_tweets=5)
printer.add_rules(tweepy.StreamRule(keyword))
printer.filter()

@Estadao @EstadaoPolitica Ora, ora,ora dessa vez o ladrão tem 10 dedos. - [{'NEG': 0.6040931344032288}]
Download Eagle Mining app at https://t.co/6V3rFRT800
Invite code:. Skydancer1 
Eagle Network is a New Cryptocurrency with an Elevated Proof of Stake (Epos) Blockchain Launched February 2021

Did you missed mining Bitcoin since 2009? Don't miss out again!

https://t.co/6V3rFRT800 - [{'NEU': 0.5883529186248779}]
RT @ceciliasotog: Si el gobierno de Chihuahua filtró el vídeo, por fin hace una primera cosa bien. Adan Augusto admite que el presidente ya… - [{'NEU': 0.837518572807312}]
RT @Mippcivzla: #NotiMippCI 📰🗞| Presidente Maduro exalta labor diplomática del Embajador de la República Popular China. Lea más ⏩ https://t… - [{'NEU': 0.9722437858581543}]
RT @5noticiasuy: 🏠 El presidente #LacallePou participó de la entrega de 43 viviendas construidas por #Mevir en Santa Rosa, Canelones | El n… - [{'NEU': 0.8860680460929871}]


ERROR:tweepy.streaming:Stream connection closed by Twitter


@nikolas_dm Poucas pessoas perceberam, Rui Falcão ao passar a palavra para o Nicolas disse: “pode falar chupetinha”, o Nicolas ouviu mas não identificou que quem disse foi o presidente da mesa. - [{'NEU': 0.9617235660552979}]
